In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col,count, size, explode, length, when

spark = SparkSession.builder \
    .appName("arxiv_eda") \
    .getOrCreate()

df = spark.read.json("../../../data/arxiv-metadata-oai-snapshot.json")

df.printSchema()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
26/01/21 09:18:38 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


root
 |-- abstract: string (nullable = true)
 |-- authors: string (nullable = true)
 |-- authors_parsed: array (nullable = true)
 |    |-- element: array (containsNull = true)
 |    |    |-- element: string (containsNull = true)
 |-- categories: string (nullable = true)
 |-- comments: string (nullable = true)
 |-- doi: string (nullable = true)
 |-- id: string (nullable = true)
 |-- journal-ref: string (nullable = true)
 |-- license: string (nullable = true)
 |-- report-no: string (nullable = true)
 |-- submitter: string (nullable = true)
 |-- title: string (nullable = true)
 |-- update_date: string (nullable = true)
 |-- versions: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- created: string (nullable = true)
 |    |    |-- version: string (nullable = true)



In [3]:
total_records = df.count()
total_records

2890332

In [4]:
from pyspark.sql.functions import sum

missing_stats = df.select([
    sum(col(c).isNull().cast("int")).alias(c)
    for c in df.columns
])


missing_stats.show(vertical=True)


-RECORD 0-----------------
 abstract       | 0       
 authors        | 0       
 authors_parsed | 0       
 categories     | 0       
 comments       | 776636  
 doi            | 1615864 
 id             | 0       
 journal-ref    | 1971387 
 license        | 452768  
 report-no      | 2702041 
 submitter      | 15186   
 title          | 0       
 update_date    | 0       
 versions       | 0       



In [13]:
missing_ratio = missing_stats.toPandas() / total_records
missing_ratio=missing_ratio.T.sort_values(by=0, ascending=False)
missing_ratio

,0
report-no,0.934855
journal-ref,0.682062
doi,0.559058
comments,0.268701
license,0.156649
submitter,0.005254
authors_parsed,0.000000
categories,0.000000
abstract,0.000000
authors,0.000000
